In [1]:
import numpy as np
from os import listdir
from os.path import isfile, join
from skimage.measure import regionprops_table
import pandas as pd
from scipy.stats import linregress
from scipy.spatial.distance import euclidean
import math
import matplotlib.pyplot as plt
import os
import json

In [2]:
def fullNucImage16(nucPath):
    scalingFactor = 10000
    for i in range(4):
        #build row block
        nuc1 =np.load(join(nucPath, "Cropped_IMG-" + str(4*i +1) + '_seg.npy'), allow_pickle=True).item()['masks']
        nuc2 =np.load(join(nucPath, "Cropped_IMG-" + str(4*i +2) + '_seg.npy'), allow_pickle=True).item()['masks']
        nuc3 =np.load(join(nucPath, "Cropped_IMG-" + str(4*i +3) + '_seg.npy'), allow_pickle=True).item()['masks']
        nuc4 =np.load(join(nucPath, "Cropped_IMG-" + str(4*i +4) + '_seg.npy'), allow_pickle=True).item()['masks']
        nuc1 = nuc1 + scalingFactor *(4*i +1)
        nuc1[nuc1 == (scalingFactor *(4*i +1))] = 0
        nuc2 = nuc2 + scalingFactor *(4*i +2)
        nuc2[nuc2 == (scalingFactor *(4*i +2))] = 0
        nuc3 = nuc3 + scalingFactor *(4*i +3)
        nuc3[nuc3 == (scalingFactor *(4*i +3))] = 0
        nuc4 = nuc4 + scalingFactor *(4*i +4)
        nuc4[nuc4 == (scalingFactor *(4*i +4))] = 0

        row = np.concatenate((nuc1, nuc2, nuc3, nuc4), axis = 1)
        if i == 0:
            nucImage = row
        else:
            nucImage = np.concatenate((nucImage, row), axis = 0)
    return nucImage

In [3]:
def getClusters(masks, nucImage):
    accuracy_value = 0.75
    clusters = list()
    dic = {}
    for mask in np.unique(masks):
        if mask != 0:
            cluster = list()
            coord = np.where(masks == mask)
            segmentationNuc = np.unique(nucImage[coord])
            for j in segmentationNuc:
                if j != 0:
                    #occurence of nucleus pixel in
                    nucOccur = np.bincount(nucImage[coord])[j]
                    #How many pixels does this nucleus have:
                    coordLen = len(np.where(nucImage == j)[0])
                    #check if most of nuclei values are in segmentation
                    if(nucOccur >= accuracy_value*coordLen):
                        cluster.append(j)
            if cluster:
                dic[mask] = cluster
                clusters.append(cluster)
    return dic
    

In [ ]:
redpath = 'segmentations/inducibleRainbow/red/5-60/aorta09'
nucPath = 'segmentations/inducibleRainbow/nuclei/5-60'
dfRotationPath = 'results/indRainbow/angles.csv'
savePath = 'results/indRainbow/anglesResult'
age = 'P5-60'
aorta = 'aorta4'

folders = [x[0] for x in os.walk(redpath)]
folders = sorted(folders)

dfAngle = pd.read_csv(dfRotationPath)
dfFiltration = dfAngle.loc[(dfAngle['age'] == age) & (dfAngle['aorta'] == aorta)]

In [ ]:
angleFlow = np.mean(dfFiltration['angle'])
angleFlowRad = math.radians(angleFlow) 
rotation = dfFiltration['rotation'].values[0]

In [ ]:
dfResult = pd.DataFrame()
for folder in folders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if "cellposeSegm" in f if "._" not in f if '.npy' in f]
    for file in files:
        #Load the segmentation masks for nuclei from the .npy file
        masks = np.load(join(folder, file), allow_pickle=True).item()['masks']
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        nucPathFile = join(nucPath, aorta)
        # create stitched image
        nucImage = fullNucImage16(nucPathFile)
        # get the clone sizes by matching inducible rainbow and nuclei segmentation
        dic = getClusters(masks, nucImage)
        if rotation == 'left rotation':
            nucImage = np.rot90(nucImage, k=1)
        elif rotation == 'right rotation':
            nucImage = np.rot90(nucImage, k=3)
        elif rotation == 'no':
            nucImage = nucImage
        # get centroid and area of all nuclei within the image
        prop_dict = regionprops_table(nucImage, properties = ('label', 'area', 'centroid'))
        df = pd.DataFrame(prop_dict)
        angles = []
        stdError = []
        mseError = [] 
        labels = []
        angleDiffs = []
        angleDiffsABS = []
        clusterSizes = []
        mseErrorBloodFlow = []
        mseBloodFlowDic = {}
        xContribution = []
        yContribution = []

        k = 0
        # iterate over all clones
        for key in dic:
            filtered_df = df[df['label'].isin(dic[key])]
            clusterSize = len(filtered_df)
            # Proceed only if the cluster has more than one element
            if clusterSize > 1:
                values = filtered_df['centroid-1'].values  # Get x-coordinates of nuclei centroids
                values.sort() # sort nuclei centroids
                distance = True
                # Check if the centroids are sufficiently spaced apart (the distance should be more than 1 pixel)
                for i in range(len(values) - 1):
                    if values[i+1] - values[i] < 1:
                        distance = False
                # If they are sufficiently spaced, perform linear regression
                if distance == True:
                    slope, intercept, r_value, p_value, std_err = linregress(x = filtered_df['centroid-1'], y= filtered_df['centroid-0'])
                    # Calculate regression angle and angle difference compared to flow angle
                    regression_angle = math.atan(slope)
                    angleDiff = math.degrees(regression_angle - angleFlowRad)
                    # Calculate predicted values based on the linear fit
                    predicted_values = slope * filtered_df['centroid-1'] + intercept
                    # Compute Euclidean distances between actual and predicted values
                    distances = [euclidean([x, y], [x, y_pred]) for x, y, y_pred in zip(filtered_df['centroid-1'], filtered_df['centroid-0'], predicted_values)]
                    mse = np.mean(distances) # Mean Squared Error of the regression
                    
                    # Calculate alternative regression based on mean position
                    sumX = sum(filtered_df['centroid-1'])
                    sumY = sum(filtered_df['centroid-0'])
                    slope2 =  math.tan(angleFlowRad)
                    intercept2 = (sumY - slope2 * sumX) / clusterSize
                    predicted_values2 = slope2 * filtered_df['centroid-1'] + intercept2
                    distances2 = [euclidean([x, y], [x, y_pred]) for x, y, y_pred in zip(filtered_df['centroid-1'], filtered_df['centroid-0'], predicted_values2)]
                    mse2 = np.mean(distances2)

                    
                    clusterSizes.append(clusterSize)
                    mseError.append(mse)
                    mseErrorBloodFlow.append(mse2)
                    stdError.append(std_err)
                    angles.append(regression_angle)
                    angleDiffs.append(angleDiff)
                    angleDiffsABS.append(abs(angleDiff))
                    labels.append(key)

                    # Calculate contributions to x distance and record MSE for blood flow
                    xDist = int(np.round(filtered_df['centroid-1'].max() - filtered_df['centroid-1'].min(), 0))
                    mseBloodFlowDic[xDist] = distances2

                    xContribution.append(xDist)
                    yContribution.append(mse2*2) 
                    k += 1


        df_help = pd.DataFrame({'age': age, 'aorta': aorta, 'label': labels, 'clusterSize': clusterSizes,
                                'angle': angles, 'bloodFlowAngle': angleFlowRad, 'angleDiff': angleDiffs, 'angleDiffABS':angleDiffsABS,
                                'mseBloodFlow': mseErrorBloodFlow,'stdError':std_err, 'mse': mseError,
                                'xContribution': xContribution, 'yContribution': yContribution})
        dfResult = pd.concat([dfResult, df_help])

dfResult.to_csv(join(savePath, age, aorta, 'angleResults_P5-60_a8.csv'), index=False)


concatenate all results together

In [41]:
####All Ages
dfList = ['results/indRainbow/anglesResult/0-5/angles0-5AllNEW.csv',
          'results/indRainbow/anglesResult/0-10/anglesP0-10AllNEW.csv',
          'results/indRainbow/anglesResult/5-10/angles5-10AllNEW.csv',
          'results/indRainbow/anglesResult/5-30/angles5-30AllNEW.csv',
          'results/indRainbow/anglesResult/5-60/angles5-60AllNEW.csv',
          'results/indRainbow/anglesResult/10-21/angles10-21AllNEW.csv',
          'results/indRainbow/anglesResult/10-30/angles10-30AllNEW.csv',
          'results/indRainbow/anglesResult/10-60/angles10-60AllNEW.csv',
          'results/indRainbow/anglesResult/21-30/angles21-30AllNEW.csv',
          'results/indRainbow/anglesResult/30-60/angles30-60AllNEW.csv']

In [42]:
dfAll = pd.DataFrame()
for path in dfList:
    df_help = pd.read_csv(path)
    dfAll = pd.concat([dfAll, df_help])

In [43]:
dfAll.to_csv('results/indRainbow/anglesResult/anglesResultsAll.csv', index = False)